In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/07 12:22:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [12]:
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

df.show()

23/08/07 12:33:05 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [13]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales_data_temp')

In [18]:
# Sample query
query = "select * from home_sales_data_temp where bedrooms > 4 and bathrooms > 3 order by date desc"
result = spark.sql(query)

In [19]:
result.show()

+--------------------+----------+----------+-------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+-------+--------+---------+-----------+--------+------+----------+----+
|741770e3-4c7d-43c...|2022-06-10|      2015|1491885|       7|        6|       5662|    9222|     1|         1|  83|
|a00a321f-ab16-490...|2022-06-07|      2016|1317548|       7|        5|       5035|   14577|     1|         1|  91|
|64eb1acb-6729-4cd...|2022-06-06|      2015|1456592|       6|        4|       5803|    8857|     3|         0|  89|
|2f3eb18d-3526-408...|2022-05-27|      2016|1352170|       6|        5|       5350|   14958|     2|         1|  81|
|bb612a79-a911-409...|2022-05-27|      2016|1459677|       6|        4|       5086|   12009|     1|         1|  78|
|41f7e212-e965-4c8...|2022-05-26|      2016|1334175|       8|        4| 

In [27]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query = """
        select year(date) as year_sold
        , round(avg(price),2) as avg_price
        from home_sales_data_temp
        where bedrooms = 4
        group by year_sold
        order by year_sold desc
        """
result = spark.sql(query)
result.show()

+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2022|296363.88|
|     2021|301819.44|
|     2020|298353.78|
|     2019| 300263.7|
+---------+---------+



In [28]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query = """
        select date_built
        , round(avg(price),2) as avg_price
        from home_sales_data_temp
        where bedrooms = 3
        and bathrooms = 3
        group by date_built
        order by date_built desc
        """
result = spark.sql(query)
result.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015| 288770.3|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [29]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
        select date_built
        , round(avg(price),2) as avg_price
        from home_sales_data_temp
        where bedrooms = 3
        and bathrooms = 3
        and floors = 2
        and sqft_living >= 2000
        group by date_built
        order by date_built desc
        """
result = spark.sql(query)
result.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [31]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
        select round(avg(price)) as avg_price_homes_above_350
        , avg(view) as avg_view_rating
        from home_sales_data_temp
        where price >= 350000
        """
result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------------+---------------+
|avg_price_homes_above_350|avg_view_rating|
+-------------------------+---------------+
|                 473796.0|         32.264|
+-------------------------+---------------+

--- 1.0604941844940186 seconds ---


Runtime is 1.06 seconds

In [33]:
# 7. Cache the the temporary table home_sales.

spark.sql("cache table home_sales_data_temp")

DataFrame[]

In [34]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_data_temp')

True

In [35]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
        select round(avg(price)) as avg_price_homes_above_350
        , avg(view) as avg_view_rating
        from home_sales_data_temp
        where price >= 350000
        """
result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------------------+---------------+
|avg_price_homes_above_350|avg_view_rating|
+-------------------------+---------------+
|                 473796.0|         32.264|
+-------------------------+---------------+

--- 0.8634040355682373 seconds ---


Runtime is 0.86 seconds
Previous, uncached runtime was 1.06 seconds

In [36]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_data_parquet")

In [37]:
# 11. Read the formatted parquet data.

df_parquet = spark. read.parquet("home_sales_data_parquet")

In [38]:
# 12. Create a temporary table for the parquet data.

df_parquet.createGlobalTempView("home_sales_data_parquet_temp")

In [39]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

query = """
        select round(avg(price)) as avg_price_homes_above_350
        , avg(view) as avg_view_rating
        from home_sales_data_temp
        where price >= 350000
        """
result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------------+---------------+
|avg_price_homes_above_350|avg_view_rating|
+-------------------------+---------------+
|                 473796.0|         32.264|
+-------------------------+---------------+

--- 0.6157078742980957 seconds ---


Runtime is 0.62 seconds.
Previous, uncached runtime was 1.06 seconds, cached runtime was 0.86 seconds

| Uncached  | Cached  |  Parquet |
|---|---|---|
|  1.06 | 0.86  | 0.62  |


In [40]:
# 14. Uncache the home_sales temporary table.

spark.sql("uncache table home_sales_data_temp")

DataFrame[]

In [41]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales_data_temp")

False

In [55]:
query = """
        select
        date_built, bedrooms, bathrooms,
        max(price) over (partition by bedrooms, bathrooms) as max_price,
        min(price) over (partition by bedrooms, bathrooms) as min_price
        from home_sales_data_temp;
        """
result = spark.sql(query)
result.show()

+----------+--------+---------+---------+---------+
|date_built|bedrooms|bathrooms|max_price|min_price|
+----------+--------+---------+---------+---------+
|      2013|       2|        2|   449985|   120023|
|      2014|       2|        2|   449985|   120023|
|      2016|       2|        2|   449985|   120023|
|      2017|       2|        2|   449985|   120023|
|      2014|       2|        2|   449985|   120023|
|      2015|       2|        2|   449985|   120023|
|      2012|       2|        2|   449985|   120023|
|      2012|       2|        2|   449985|   120023|
|      2017|       2|        2|   449985|   120023|
|      2012|       2|        2|   449985|   120023|
|      2014|       2|        2|   449985|   120023|
|      2016|       2|        2|   449985|   120023|
|      2013|       2|        2|   449985|   120023|
|      2017|       2|        2|   449985|   120023|
|      2017|       2|        2|   449985|   120023|
|      2012|       2|        2|   449985|   120023|
|      2012|